In [2]:
from openai import OpenAI
import os
import requests

In [4]:
client = OpenAI(
    api_key = os.environ.get('OPENAI_API_KEY')
)

File

In [5]:

api_endpoint = "https://api.openai.com/v1/vector_stores"
api_key = os.environ.get('OPENAI_API_KEY')

# 定義請求標頭
headers = {
    "Authorization": f"Bearer {api_key}",
    "OpenAI-Beta": "assistants=v2"
}

# 發送GET請求以獲取所有的vector stores
response = requests.get(api_endpoint, headers=headers)

# 檢查請求是否成功
if response.status_code == 200:
    vector_stores = response.json()
    # 列出所有的vector stores
    for store in vector_stores['data']:
        print(f"ID: {store['id']}, Name: {store['name']}")
else:
    print(f"Failed to retrieve vector stores: {response.status_code} {response.text}")



ID: vs_ZbiGyuz5pe1HnV6xXZ5HXPBS, Name: Algorithm


In [6]:
api_endpoint2 = "https://api.openai.com/v1/files"

response2 = requests.get(api_endpoint2, headers=headers)

if response2.status_code == 200:
    vector_stores = response2.json()
    # 列出所有的vector stores
    for store in vector_stores['data']:
        print(f"ID: {store['id']}, Name: {store['filename']}")
else:
    print(f"Failed to retrieve vector stores: {response2.status_code} {response2.text}")

ID: file-W0SutWWSidJoAqCnnYl5unXx, Name: 改良QuickSort作業說明_更正.pdf
ID: file-2dUC8dbXr1pOPzcfiNqYCQEH, Name: Eclipse安裝與輸出說明.pdf


In [7]:
vector_store = client.beta.vector_stores.create(name="Algorithm")

# file_paths = ["改良QuickSort作業說明_更正.pdf","Eclipse安裝與輸出說明.pdf","chapter5.pdf"]
file_paths = ["改良QuickSort作業說明_更正.pdf","Eclipse安裝與輸出說明.pdf"]
file_streams = [open(file_path, "rb") for file_path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


Assistant

In [9]:
assistant = client.beta.assistants.create(
  name="Algorithm",
  instructions="1. You are now an assistant for the professor in algorithm class, you need to teach the class in English 2. Read pdf's content and give me an English script for class 3. Use the knowledge only in the pdf, else don't answer and say you don't know",
  model="gpt-4o",
  # tools=[{"type": "file_search"}, {"type": "vision"}],
  tools=[{"type": "file_search"}],
)

In [10]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

Thread

In [11]:
thread = client.beta.threads.create()

thread

Thread(id='thread_YM7JIACmu9loXnMKcTNciCUS', created_at=1723869517, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [15]:
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "briefly explain 改良QuickSort作業說明_更正.pdf."
    }
  ]
)


print(thread.tool_resources.file_search)

None


In [16]:
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

In [17]:
messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
# print(messages)
print(messages[0].content[0].text.value)

The document "改良QuickSort作業說明_更正.pdf" provides detailed instructions for an assignment aimed at improving the efficiency of the standard QuickSort algorithm by integrating it with Insertion Sort. Here is a brief explanation suitable for a class presentation:

---

### Script for Class

---

**Introduction:**

Today, we will discuss how to enhance the QuickSort algorithm by incorporating the Insertion Sort to improve its efficiency.

---

**Assignment Overview:**

Your task is to implement the pseudocode for QuickSort from the textbook in Java and enhance it by integrating Insertion Sort. You will practice converting pseudocode into actual code while improving the QuickSort algorithm's efficiency.

---

**Requirements:**

1. **Programming Language and Environment:**
   - Use Java for implementation.
   - The project must be compatible with JavaSE-17.

2. **Algorithm Integration:**
   - Implement the QuickSort algorithm using the provided pseudocode.
   - Use Insertion Sort for subarrays

In [ ]:
# from django.shortcuts import render
# import requests
# from django.http import JsonResponse, HttpRequest
# from openai import OpenAI
# import os
# from PyPDF2 import PdfReader

# # Create your views here.

# client = OpenAI(
#     api_key = os.environ.get('OPENAI_API_KEY')
# )



# def gpt_read_pdf(request):
#     try:
#         page_numbers = range(1, 6)  
#         pdf_content = read_pdf_pages("D:\\中央大學\\第6學期\\專題\\project\\EMI_helper\\ncu_emi\\gpt\\[中央大學]20240417創造社群與自媒體影響力的經營策略(列印版).pdf", page_numbers)

#         completion = client.chat_completions.create(
#             model="gpt-3.5-turbo",
#             messages=[
#                 {"role": "system", "content": "read pdf's content and give me an English script for class:"},
#                 {"role": "user", "content": pdf_content}
#             ]
#         )
        
#         reply = completion['choices'][0]['message']['content']
#         return JsonResponse({'reply': reply})
#     except Exception as e:
#         return JsonResponse({'error': str(e)}, status=500)


# def read_pdf_pages(file_path, pages):

#     reader = PdfReader(file_path)
#     number_of_pages = len(reader.pages)
#     content = ""

#     for page in pages:
#         if page <= number_of_pages:
#             content += reader.pages[page - 1].extract_text()
#         else:
#             content += f"Page {page} does not exist in the PDF"
    
#     return content


            

    










